In [21]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.applications.xception import Xception
from pickle import load
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [22]:
def extract_features(filename, model):
    try:
        image = Image.open(filename)
    except:
        print("Error occurred!")
    image = image.resize((299, 299))
    image = np.array(image)
    # if image has 4 channels, convert them to 3 channels
    if image.shape[2]==4:
        image = image[..., :3]
    image = np.expand_dims(image, axis=0)
    image = image/127.5
    image = image - 1.0
    feature = model.predict(image)
    return feature

In [23]:
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index==integer:
            return word
    return None

In [27]:
def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'start'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen = max_length)
        pred = model.predict([photo, sequence], verbose=0)
        pred = np.argmax(pred)
        word = word_for_id(pred, tokenizer)
        if word is None:
            break
        in_text += ' '+word
        if word=='end':
            break
    return in_text

In [28]:
max_length = 32
tokenizer = load(open("tokenizer.p", "rb"))
model = load_model('model.h5')
xception_model = Xception(include_top = False, pooling="avg")

In [36]:
img_path = 'C:\\Users\\Tushar\\Desktop\\DL Projects\\ImageCaptionGenerator\\test_data\\5.jpg'
photo = extract_features(img_path, xception_model)
description = generate_desc(model, tokenizer, photo, max_length)
print(' '.join(description.split()[1:-1]).capitalize()+'.')

Baseball player in red and white uniform is playing baseball.
